In [1]:
from __future__ import print_function
import ipywidgets as widgets 
import matplotlib.pyplot as plt
import numpy as np

# JC Model

### Job Creator parameters

$r = instructionCount*JO.rate + bandwidthUsage*JO.rate$ : The reward the JC is willing to pay to have a job executed.

$pID = 1$ : The probability that JC will identify a job as class 1, 2, or junk.

$p_{v1}$ : probability the JC verifies answers from class 1. 

$p_{v2}$ : probability the JC verifies answers from class 2. This probability is 0 because it can just go straight to mediation. 

$vc$ : cost to JC to verify result.

$b$ : the value or benefit JC gets when a job is performed correctly.

$matchPrice$ : the amount JC pays to Solver for finding a suitable RP. 

$mediatorPrice$ : The amount JC pays to mediator for being available. 

$p_1$ : probability that job returns answer in class $a1$

$p_2 = 1-p_1$ : probability that job returns answer in class $a2$

$deposit = price\times PR\times n$ : the amount the Job Creator must post as a security deposit in order to get matched. It multiplied by the $n$, which as a reminder is the number of times the mediator will replicate a job. 

In [2]:
def JCU(n,PR,NDPR,  b,mb,r,p1,pID,idc,pv1,vc1,pv2,vc2,gasCost,  pq):
    '''if (1-pq) then 1=p1+p2+p3. p1 and p2 would how likely the RP is guess a valid looking solution. If the ID task is good they will be 0, so we assume p3=1. This means we only need one set of (1-pq) terms'''
    
    p2 = 1-p1
    
    NDdeposit = r * NDPR + (r*n)
    
    honest = pID*p1*pv1*pq * (b-r-vc1-idc)
    
    # I'm not sure if these two terms are relevant or not. When would they come into play?
    # I guess they would be relevant when pID doesn't work? But if it doesn't work, just send to mediation. I think DRP covers this adequately 
#     HJC_DRP_Gain = pID*p1*pv1*(1-pq)* (p1**n) * (mb+r-vc1-idc)
#     HJC_DRP_Lose = pID*p1*pv1*(1-pq)* (1-p1**n) * (-NDdeposit - gasCost -vc1-idc)
 
    
    lazy   = pID*p1*(1-pv1)*pq * (b - r - idc)
    
    
    normal = honest + lazy
    
    # ----------------------------------------------------
    # pv2 will be 0, since the point is get jobs done this way for free.
    # ----------------------------------------------------
#     CNDJC_HRP_Gain = p2*pv2*pq * (p1**n) * (b + r - vc2)     
#     CNDJC_HRP_Lose = p2*pv2*pq * (1-p1**n) * (b - NDdeposit - vc2)
    
#     CNDJC_DRP_Gain = p2*pv2*(1-pq) * (p1**n) * (mb + r - vc2)     
#     CNDJC_DRP_Lose = p2*pv2*(1-pq) * (1-p1**n) * ( - NDdeposit - vc2)
    
    NDJC_HRP_Gain = pID*p2*(1-pv2)*pq   * (p1**n) * (b + r - idc)    # gasCost is refunded  
    NDJC_HRP_Lose = pID*p2*(1-pv2)*(pq) * (1-p1**n) * (b - NDdeposit - gasCost - idc)
    
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose
    
    # ----------------------------------------------------------
    # 1-pq is likely not p1 or p2 so deal with it separately?
    # ----------------------------------------------------------
#     NDJC_DRP_Gain = p2*(1-pv2)*(1-pq) * (p1**n) * (mb + r)  
#     NDJC_DRP_Lose = p2*(1-pv2)*(1-pq) * (p1**n) * ( - NDdeposit)
    
    DRP_Gain = pID*(1-pq) * (p1**n) * (mb + r - idc)
    DRP_Lose = pID*(1-pq) * (1-p1**n) * (- NDdeposit - gasCost - idc)      
    DRP_scared = pID*(1-pq) * (-r - idc)
    
    if p1 != 1:
        '''JC is non-deterministic'''
        DRP = max(DRP_Gain+DRP_Lose, DRP_scared)
    else: 
        '''JC is deterministic'''
        DRP = DRP_Gain 
#     DRP = max(DRP_Gain+DRP_Lose, DRP_scared)
    
    
    noID_HRP   = (1-pID) * pq * (b-r)
    noID_DRP = (1-pID) * (1-pq) * (-r)
    noID = noID_HRP + noID_DRP
    
    return normal + ND + DRP + noID

 
    

In [3]:
def RPU(n,PR,NDPR,  r,p1,pID,pv1,pv2,  pq,ec,I,dc):
    p2 = 1-p1
    deposit = r * PR + (r*n)
        
    honestJC = pq*pID*p1*pv1*(r-ec) #           
    lazyJC = pq*pID*p1*(1-pv1)*(r-ec)
    normal = honestJC + lazyJC    
    
    NDJC_HRP_Gain = pID*p2*(1-pv2)*(pq) * (1-p1**n) * (r-ec)#
    NDJC_HRP_Lose = pID*p2*(1-pv2)*(pq) * (p1**n) * (-ec - deposit)#
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose    
    
    DRP_Gain = pID*(1-pq) * (1-p1**n) * (r-dc)  #
    DRP_Lose = pID*(1-pq) * (p1**n) * (-dc - deposit)#
    DRP_scared = pID*(1-pq) * (r - dc) #
#     DRP = min(DRP_Gain+DRP_Lose, DRP_scared) #this isn't quite right, it won't be decided by the RPU. Its decided by the JCU
    DRP = DRP_Gain+DRP_Lose
    
    
    noID_HRP = (1-pID) * pq * (r - ec)#
    noID_DRP = (1-pID) * (1-pq) * (r - dc)#
    noID = noID_HRP + noID_DRP
    
    return normal + ND + DRP + noID + I
    

In [4]:
def RPNDUD(n,PR,NDPR,  r,p1,pID=1,  q=1, ec=0, I=0,dc=0):
    p2 = 1-p1
#     deposit = price * PR
#     deposit = price * PR * n
    deposit = r * PR + (r * n)
    
    
    execCost = ec
    
    insurance =  I

    #Cj is cost of execution for fraudulant job, picking a random number or "close" value
    fraudCost = dc
    
    honestJC = pID*q*p1*(r-execCost) #compute answer 1 (p1) correctly (q) receive reward (price)
    
    NDJC_HRP_Gain = pID*q*p2*(1-p1**n)*(r-execCost) #compute answer 2 (p2) correctly (q) get sent to mediation, who finds nondeterminism (1-p1^n) and recieve reward (r) and nonDet bonus (f2-f)*k kickback=k
    NDJC_HRP_Lose = pID*q*p2*(p1**n)*(-deposit-execCost) # compute answer 2 correctly (p2) get sent to mediation, who doesn't find nondeterminsm (p1^n) and get fined (f)
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose 
    
    DRP_Gain = pID*(1-q)*(1-p1**n)*(r-fraudCost)# compute incorrectly (1-q) get sent to mediation, finds nondeterminism (1-p1^n) get compensation.
    DRP_Lose = pID*(1-q)*(p1**n)*(-deposit-fraudCost) #compute incorrectly (1-q) get sent to mediation, who doesn't find nondeterminsm (p1^n) and get fined (f)
    DRP = DRP_Gain+DRP_Lose
    
    DRP_scared  = pID*(1-q)*(r-fraudCost) # JC doesn't want to risk getting caught, just pay RP. 
    
    #     noVerifypayout = (1-p)*price #compute answer 1 (p1) correctly (q) receive reward (price)
    noID_HRP = (1-pID) * q * (r - execCost)
    noID_DRP = (1-pID) * (1-q) * (r - fraudCost)
    noVerifypayout = noID_HRP + noID_DRP
    
    
    RCnonDet = honestJC + ND + DRP + insurance + noVerifypayout
    return RCnonDet

In [5]:
def getQ(ec,dc,f,m,r,p):
    qPoint = (ec-dc)/(f)
    if (p>qPoint):
        return(1)
    else:
        return(0)

In [6]:
def plot(xs,ys):
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    
    xlabel = next(iter(xs))
    for y in ys.keys():
        plt.plot(xs[xlabel],ys[y],label=y)
    
    plt.xlabel(xlabel)
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    

In [7]:
x,step = np.linspace(0,1,51,retstep=True)

def Util(vary, n,PR,NDPR,  b,mb,r,p1,pID,idc,pv1,vc1,pv2,vc2,  pq,dc,ir,roi):       
    
    # JC y-axis
    ys ={}
    
    # RP y-axis
    ec = r*(1-roi) # execution cost
    deposit = r * PR + (r*n)
    I =  deposit*ir # insurance    
    
    if vary == 'p1':
        ys['yNDJC'] = list(map(lambda p1: JCU(n,PR,NDPR, b=b,mb=mb,r=r,p1=p1,pID=pID,idc=idc,pv1=pv1,vc1=vc1,pv2=pv2,vc2=vc2,gasCost=JCgasCost, pq=pq),x))
        ys['yHJC'] =  list(map(lambda p1: JCU(n,PR,NDPR, b=b,mb=mb,r=r,p1=1,pID=pID,idc=idc,pv1=1,vc1=0,pv2=1,vc2=0,gasCost=JCgasCost, pq=pq),x))       

        ys['yHRP'] = list(map(lambda p1: RPU(n,PR,NDPR,  r,p1,pID,pv1,pv2,  pq=1,ec=ec,I=I,dc=dc),x))
        ys['yDRP'] = list(map(lambda p1: RPU(n,PR,NDPR,  r,p1,pID,pv1,pv2,  pq,ec,I,dc),x))
        xs = {'P(a1)':x}
        
    if vary == 'q':
#         ys['yNDJC'] = list(map(lambda pq: JCU(n,PR,NDPR, b=b,mb=mb,r=r,p1=p1,pID=pID,idc=idc,pv1=pv1,vc1=vc1,pv2=pv2,vc2=vc2,gasCost=JCgasCost, pq=pq),x))
#         ys['yHJC'] =  list(map(lambda pq: JCU(n,PR,NDPR, b=b,mb=mb,r=r,p1=1,pID=pID,idc=idc,pv1=1,vc1=0,pv2=1,vc2=0,gasCost=JCgasCost, pq=pq),x))       
        ys['RPNDUD'] = list(map(lambda pq: RPNDUD(n,PR,NDPR,  r,p1,pID,          q=pq, ec=ec, I=I,dc=dc),x))
        print("n,PR,NDPR,  r,p1,pID,          q=pq, ec=ec, I=I,dc=dc")
        print(n,PR,NDPR,  r,p1,pID,          "pq", ec, I,dc)
        ys['yDRP']   = list(map(lambda pq:    RPU(n,PR,NDPR,  r,p1,pID,pv1,pv2,  pq,ec,I,dc),x))
        print("n,PR,NDPR,  r,p1,pID,pv1,pv2,  pq,ec,I,dc")
        print(n,PR,NDPR,  r,p1,pID,pv1,pv2,  pq,ec,I,dc)
        xs = {'P(q)':x}
        
    if vary == 'pID':
        ys['yNDJC'] = list(map(lambda pID: JCU(n,PR,NDPR, b=b,mb=mb,r=r,p1=p1,pID=pID,idc=idc,pv1=pv1,vc1=vc1,pv2=pv2,vc2=vc2,gasCost=JCgasCost, pq=pq),x))
        ys['yHJC'] =  list(map(lambda pID: JCU(n,PR,NDPR, b=b,mb=mb,r=r,p1=1,pID=pID,idc=idc,pv1=1,vc1=0,pv2=1,vc2=0,gasCost=JCgasCost, pq=pq),x))       

        ys['yHRP'] = list(map(lambda pID: RPU(n,PR,NDPR,  r,p1,pID,pv1,pv2,  pq=1,ec=ec,I=I,dc=dc),x))
        ys['yDRP'] = list(map(lambda pID: RPU(n,PR,NDPR,  r,p1,pID,pv1,pv2,  pq,ec,I,dc),x))
        xs = {'P(ID)':x}
        
    
    
    plot(xs,ys)

In [8]:
# Vary
wvary = widgets.Dropdown(options=['p1','q','pID'])

# Mediator parameters
wn = widgets.IntSlider(min=0,max=10,step=1,value=1, description='n',continuous_update=False)
wPR = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='PR', continuous_update=False) #Penalty Rate
wNDPR = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='NDPR',continuous_update=False)#Non-deterministic Penalty rate

# JC parameters
wb = widgets.IntSlider(min=0,max=10000,step=1,value=1, description='b',continuous_update=False)
wmb = widgets.IntSlider(min=0,max=10000,step=1,value=1, description='mb',continuous_update=False)
wr = widgets.IntSlider(min=0,max=10000,step=1,value=1, description='r',continuous_update=False)
wp1 = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='p1', continuous_update=False)
wpID = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='P(ID)', continuous_update=False)
widc = widgets.IntSlider(min=0,max=1000,step=1,value=0, description='idc',continuous_update=False)
wpv1 = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='P(verify 1)', continuous_update=False) # how often JC verifies
wvc1 = widgets.FloatSlider(min=0,max=100,step=.1,value=0, description='vc1',continuous_update=False) # cost for JC to verify
wpv2 = widgets.FloatSlider(min=0,max=1,step=.1,value=0, description='P(verify 2)', continuous_update=False) # how often JC verifies
wvc2 = widgets.FloatSlider(min=0,max=100,step=.1,value=0, description='vc2',continuous_update=False) # cost for JC to verify

JCgasCost = 0
JCmatchCost = 1
JCmediatorCost = 1

# RP parameters
wpq = widgets.FloatSlider(min=0,max=1,step=.1,value=1,description='pq', continuous_update=False) # probability that RP runs correctly
wdc= widgets.FloatSlider(min=0,max=100,step=.1,value=4, description='dc',continuous_update=False) # cost for RP to generate fake job response
wir = widgets.FloatSlider(min=0,max=1,step=.01,value=.5 ,description='ir',continuous_update=False) # % of deposit that can be recovered because execution is so cheap.
wroi = widgets.FloatSlider(min=0,max=1,step=.01,value=.5,description='roi', continuous_update=False) # what percent of reward is profit


RPgasCost = 0
RPmatchCost = 1
RPmediatorCost = 1

# ui = widgets.HBox([wn,wPR,wNDPR,wPrice,wp,wcj,wq,wroi,winsurance])
ui0 = widgets.HBox([wvary])
ui1 = widgets.HBox([wn,wPR,wNDPR])
ui2 = widgets.HBox([wb,wmb,wr,wpID,widc])
ui22 = widgets.HBox([wp1,wpv1,wvc1,wpv2,wvc2])
ui3 = widgets.HBox([wpq,wroi,wir,wdc])

plotParams = {'vary':wvary,
              'n': wn,
              'PR':wPR,
              'NDPR':wNDPR,
              'b':wb,
              'mb':wmb,
              'r':wr,
              'p1':wp1,
              'pID':wpID,
              'idc':widc,
              'pv1':wpv1,
              'vc1':wvc1,
              'pv2':wpv2,
              'vc2':wvc2,
              'pq': wpq,
              'dc': wdc,
              'ir': wir,
              'roi':wroi}



### Honest JC, Deceitful RP
p1 = 1
pv1 = 1   
vc1 = 0
pv2 = 0
pc2 = 0

$p_{ID}*pq*(b-r-idc) + (1-pID)*pq*(b-r)  + p_{ID}*(1-pq)*(mb + r - idc) + (1-p_{ID})*(1-pq)*(-r)$

**Simplify**

$pq * (b-r -p_{ID}*idc)  + (1-pq)*(p_{ID}*(mb + r - idc) + (1-p_{ID})*(-r))$


In [9]:
interactive_plot = widgets.interactive_output(Util,plotParams);
# output = interactive_plot.children[-1]
interactive_plot.layout.height = '300px'
display(interactive_plot,ui1,ui2,ui22,ui3,ui0)

Output(layout=Layout(height='300px'))